In [47]:
import numpy as np
import sbox as sb

KEY_SIZE = 128
WORD_ARRAY_SIZE = 4
WORD_SIZE = int(KEY_SIZE / WORD_ARRAY_SIZE)

In [48]:
def stringToHex(string: str) -> list:
  """
  Converts a string to a list of hex values
  """
  return [hex(ord(c)) for c in string]

In [49]:
test = stringToHex("Thats my Kung Fu")
print(test, type(test))

['0x54', '0x68', '0x61', '0x74', '0x73', '0x20', '0x6d', '0x79', '0x20', '0x4b', '0x75', '0x6e', '0x67', '0x20', '0x46', '0x75'] <class 'list'>


In [50]:
def keyToMatrix(key: list) -> list:
  """
  Converts a list of hex values to a 4x4 matrix
  """
  return np.matrix(np.reshape(key, (WORD_ARRAY_SIZE, int(KEY_SIZE / WORD_SIZE)))).tolist()

In [51]:
testMatrix = keyToMatrix(test)
print(testMatrix, type(testMatrix))

[['0x54', '0x68', '0x61', '0x74'], ['0x73', '0x20', '0x6d', '0x79'], ['0x20', '0x4b', '0x75', '0x6e'], ['0x67', '0x20', '0x46', '0x75']] <class 'list'>


In [52]:
def g(key: list) -> list:
  """
  Performs the g() operation on a key
  1. Rotates the key by 1 byte
  2. Applies the sbox to each byte
  3. Add the round constant (XOR)
  """
  key = np.concatenate((key[1:], key[:1]))
  key = [hex(sb.Sbox[int(x, 16)]) for x in key]
  key[0] = hex(int(key[0], 16) ^ 0x01)

  return key

In [53]:
testG = g(testMatrix[3])
print(testG, type(testG))

['0xb6', '0x5a', '0x9d', '0x85'] <class 'list'>


In [54]:
def xor(key1, key2):
  """
  XORs two keys
  """
  key = []
  key1 = np.matrix(key1).tolist()[0]
  key2 = np.matrix(key2).tolist()[0]
  
  for i in range(WORD_ARRAY_SIZE):
    # print(key1[i], type(key1[i]), key2[i], type(key2[i]))
    key.append(hex(int(key1[i], 16) ^ int(key2[i], 16)))
    
  return key

In [55]:
keyXor = xor(testMatrix[0], testG)
print(keyXor, type(keyXor))

['0xe2', '0x32', '0xfc', '0xf1'] <class 'list'>


In [56]:
def nextRoundKey(key: list) -> list:
  """
  Generates the next round's key from the previous round's key
  """
  # print("init key:", key, type(key))
  # print("xor", xor(key[0], g(key[3])))
  # key = np.array(key)
  w = [xor(key[0], g(key[3]))]
  
  # print("w:", w)
  for i in range(1, WORD_ARRAY_SIZE):
    # print("in for loop: ", i)
    # print(f"w[{i-1}]:", w[i-1], type(w[i-1]))
    # print(f"key[{i}]:", key[i], type(key[i]))
    
    newKey = xor(w[i - 1], key[i])
    # print("newKey:", newKey, type(newKey))
    
    w.append(newKey)
    # np.append(w, xor(w[i - 1], key[i]))
    
  return w

In [58]:
for i in range(11):
  print(f"Round {i}:", testMatrix)
  testMatrix = nextRoundKey(testMatrix)
  print()

Round 0: [['0x54', '0x68', '0x61', '0x74'], ['0x73', '0x20', '0x6d', '0x79'], ['0x20', '0x4b', '0x75', '0x6e'], ['0x67', '0x20', '0x46', '0x75']]

Round 1: [['0xe2', '0x32', '0xfc', '0xf1'], ['0x91', '0x12', '0x91', '0x88'], ['0xb1', '0x59', '0xe4', '0xe6'], ['0xd6', '0x79', '0xa2', '0x93']]

Round 2: [['0x55', '0x8', '0x20', '0x7'], ['0xc4', '0x1a', '0xb1', '0x8f'], ['0x75', '0x43', '0x55', '0x69'], ['0xa3', '0x3a', '0xf7', '0xfa']]

Round 3: [['0xd4', '0x60', '0xd', '0xd'], ['0x10', '0x7a', '0xbc', '0x82'], ['0x65', '0x39', '0xe9', '0xeb'], ['0xc6', '0x3', '0x1e', '0x11']]

Round 4: [['0xae', '0x12', '0x8f', '0xb9'], ['0xbe', '0x68', '0x33', '0x3b'], ['0xdb', '0x51', '0xda', '0xd0'], ['0x1d', '0x52', '0xc4', '0xc1']]

Round 5: [['0xaf', '0xe', '0xf7', '0x1d'], ['0x11', '0x66', '0xc4', '0x26'], ['0xca', '0x37', '0x1e', '0xf6'], ['0xd7', '0x65', '0xda', '0x37']]

Round 6: [['0xe3', '0x59', '0x6d', '0x13'], ['0xf2', '0x3f', '0xa9', '0x35'], ['0x38', '0x8', '0xb7', '0xc3'], ['0xef', '0x6